In [130]:
import json
import matplotlib.pyplot as plt
import seaborn as sns

In [131]:
#TODO: concat these and write to one file.
with open('../data/flutter/flutter_issues_labeled_1.json') as json_data:
    data1 = json.load(json_data)
with open('../data/flutter/flutter_issues_labeled_2.json') as json_data:
    data2 = json.load(json_data)
with open('../data/flutter/flutter_issues_labeled_3.json') as json_data:
    data3 = json.load(json_data)
with open('../data/flutter/flutter_issues_labeled_4.json') as json_data:
    data4 = json.load(json_data)
with open('../data/flutter/flutter_issues_labeled_5.json') as json_data:
    data5 = json.load(json_data)
data1 = [issue for issue in data1 if 'training_labels' in issue]
data2 = [issue for issue in data2 if 'training_labels' in issue]
data3 = [issue for issue in data3 if 'training_labels' in issue]
data4 = [issue for issue in data4 if 'training_labels' in issue]
data5 = [issue for issue in data5 if 'training_labels' in issue]
print(data5[0])
data = data1 + data2 + data3 + data4 + data5

{'url': 'https://api.github.com/repos/flutter/flutter/issues/8235', 'repository_url': 'https://api.github.com/repos/flutter/flutter', 'labels_url': 'https://api.github.com/repos/flutter/flutter/issues/8235/labels{/name}', 'comments_url': 'https://api.github.com/repos/flutter/flutter/issues/8235/comments', 'events_url': 'https://api.github.com/repos/flutter/flutter/issues/8235/events', 'html_url': 'https://github.com/flutter/flutter/issues/8235', 'id': 208313759, 'number': 8235, 'title': 'Provide a constant for the padding in a material ListView', 'user': {'login': 'Hixie', 'id': 551196, 'avatar_url': 'https://avatars2.githubusercontent.com/u/551196?v=4', 'gravatar_id': '', 'url': 'https://api.github.com/users/Hixie', 'html_url': 'https://github.com/Hixie', 'followers_url': 'https://api.github.com/users/Hixie/followers', 'following_url': 'https://api.github.com/users/Hixie/following{/other_user}', 'gists_url': 'https://api.github.com/users/Hixie/gists{/gist_id}', 'starred_url': 'https:/

TODO: concat the above dicts into one file...

In [132]:
num_positive_labels = 0
total_num_labels = 0
for issue in data:
    for assignee in issue['training_labels']:
        if issue['training_labels'][assignee] == 1:
            num_positive_labels += 1
        total_num_labels += 1
print("num positive events: " + str(num_positive_labels))
print("num total events: " + str(total_num_labels))

num positive events: 713
num total events: 2434


Now filter out any issues that never end up getting a commit/PR before they are closed. Idea being that these are not 'real' issues.

In [133]:
num_positive_labels = 0
total_num_labels = 0
for issue in data:
    for assignee in issue['training_labels']:
        if len(issue['matching_prs']) != 0 or len(issue['matching_commits']) != 0:
            if issue['training_labels'][assignee] == 1:
                num_positive_labels += 1
            total_num_labels += 1
print("num positive events: " + str(num_positive_labels))
print("num total events: " + str(total_num_labels))

num positive events: 713
num total events: 809


In [134]:
person_counts = {}
person_totals = {}
for issue in data:
    for assignee in issue['training_labels']:
        if len(issue['matching_prs']) != 0 or len(issue['matching_commits']) != 0:
            if issue['training_labels'][assignee] == 1:
                person_counts[assignee] = person_counts.get(assignee, 0) + 1
            person_totals[assignee] = person_totals.get(assignee, 0) + 1
person_percents = {}
for person in person_totals:
    person_percents[person] = person_counts.get(person, 0) / person_totals[person]
    
print("Number of successful tasks: ")
print(person_counts)
print("Number of total tasks: ")
print(person_totals)
print("Percentage of tasks completed: ")
print(person_percents)

Number of successful tasks: 
{'Hixie': 85, 'abarth': 414, 'collinjackson': 12, 'HansMuller': 63, 'devoncarew': 1, 'mpcomplete': 20, 'yjbanov': 10, 'krisgiesing': 4, 'danrubel': 8, 'xster': 3, 'sethladd': 4, 'jason-simmons': 13, 'dragostis': 5, 'jimbeveridge': 1, 'tvolkert': 10, 'pq': 6, 'aghassemi': 3, 'jakobr-google': 24, 'chinmaygarde': 1, 'gspencergoog': 3, 'dvdwasibi': 1, 'cbracken': 3, 'johnmccutchan': 9, 'goderbauer': 2, 'lequem': 1, 'mehmetf': 2, 'sivachandra': 1, 'mravn-google': 3, 'jcollins-g': 1}
Number of total tasks: 
{'abarth': 433, 'eseidelGoogle': 2, 'Hixie': 103, 'collinjackson': 17, 'HansMuller': 73, 'devoncarew': 2, 'vlidholt': 2, 'mpcomplete': 22, 'yjbanov': 11, 'krisgiesing': 5, 'danrubel': 9, 'xster': 5, 'sethladd': 8, 'jason-simmons': 14, 'aghassemi': 7, 'dragostis': 7, 'jimbeveridge': 1, 'tvolkert': 11, 'chinmaygarde': 7, 'pq': 6, 'jakobr-google': 24, 'johnmccutchan': 10, 'qchong': 1, 'cbracken': 6, 'gspencergoog': 3, 'dvdwasibi': 1, 'mit-mit': 2, 'goderbauer': 3

In [135]:
#TODO: compare characteristics of issues that are successfully completed.